<a href="https://colab.research.google.com/github/valerymamontov/colab_notebooks/blob/main/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RFM-сегментация пользователей  
  
Для сегментации использовался тестовый набор, взятый в Интернете.  
Датасет и описание - [ссылка](https://archive.ics.uci.edu/ml/datasets/online+retail#)  
Загрузил себе в ВК - [ссылка](https://vk.com/doc526127569_581872137)  
Данные датированы 2010-2011 годом.  
Для обработки они загружены в бесплатный кластер DataBricks.  

In [ ]:
# загрузка в databricks файла с данными выполнена через GUI
# после загрузки файл попадет в папку dbfs:/FileStore/tables
# такой путь нельзя использовать в pandas (возвращает ошибку, что файл не найден)
# требуется скопировать файл в другой каталог, например, в file:/databricks/driver/
# ссылка на документацию - https://kb.databricks.com/python/dbfs-file-size-limit.html

# команды, вводить каждую в отдельной строке

# %fs ls "dbfs:/FileStore/tables" - проверка файла после загрузки
# %fs cp dbfs:/FileStore/tables/online_retail.csv file:/databricks/driver/data.csv - копирование файла
# %fs ls "file:/databricks/driver/" - проверка после копирования

In [ ]:
# %fs cp dbfs:/FileStore/tables/online_retail.csv file:/databricks/driver/data.csv

res0: Boolean = true

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# ссылка на файл и схема (описание полей)
# url = "https://vk.com/doc526127569_581872137"
file = "file:/databricks/driver/data.csv"

schema = {
    'InvoiceNo': np.str, 
    'StockCode': np.str,
    'Description': np.str, 
    'Quantity': np.int64, 
    'InvoiceDate': np.str,
    'UnitPrice': np.float64, 
    'CustomerID': np.float64, 
    'Country': np.str,
}

In [ ]:
# создание датафрейма
df = pd.read_csv(file, dtype=schema, delimiter=",")
df.head()

Out[62]:

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 8:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 8:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom


In [ ]:
# анализ датафрейма на наличие пустых значений
df.isnull().sum()

Out[63]: InvoiceNo 0
StockCode 0
Description 1454
Quantity 0
InvoiceDate 0
UnitPrice 0
CustomerID 135080
Country 0
dtype: int64

In [ ]:
# удаление строк с пустыми значениями в столбце CustomerID
df.dropna(subset=["CustomerID"], inplace=True)

In [ ]:
# преобразование столбца InvoiceDate в дату
df["InvoiceDate2"] = df.InvoiceDate.apply(lambda x: datetime.strptime(x.split(" ")[0], "%d.%m.%Y"))

In [ ]:
# как оказалось, в столбце Quantity (количество) присутствует большое число отрицательных чисел
df.query("Quantity < 0")

Out[66]:

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDate2
141,C536379,D,Discount,-1,01.12.2010 9:41:00,27.50,14527.0,United Kingdom,2010-12-01
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01.12.2010 9:49:00,4.65,15311.0,United Kingdom,2010-12-01
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01.12.2010 10:24:00,1.65,17548.0,United Kingdom,2010-12-01
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01.12.2010 10:24:00,0.29,17548.0,United Kingdom,2010-12-01
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01.12.2010 10:24:00,0.29,17548.0,United Kingdom,2010-12-01
...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,09.12.2011 9:57:00,0.83,14397.0,United Kingdom,2011-12-09
541541,C581499,M,Manual,-1,09.12.2011 10:28:00,224.69,15498.0,United Kingdom,2011-12-09
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,09.12.2011 11:57:00,10.95,15311.0,United Kingdom,2011-12-09
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,09.12.2011 11:58:00,1.25,17315.0,United Kingdom,2011-12-09


In [ ]:
# очевидно это отмененные заказы
df.query("CustomerID == 12346.0")

Out[67]:

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDate2
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,18.01.2011 10:01:00,1.04,12346.0,United Kingdom,2011-01-18
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,18.01.2011 10:17:00,1.04,12346.0,United Kingdom,2011-01-18


In [ ]:
# при группировке такие заказы схлопнутся, в сумме дадут ноль
# однако, если столбец Quantity в сумме даст ноль, то столбец  UnitPrice - нет

# для решения этой проблемы будет добавлен новый столбец (Quantity*UnitPrice)
df["Price"] = df["Quantity"] * df["UnitPrice"]
df.head()

Out[68]:

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDate2,Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 8:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 8:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 8:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [ ]:
# группировка данных (из датафрейма отбираются только нужные столбцы)
orders = df.groupby(by=["InvoiceNo", "CustomerID", "InvoiceDate2"]).agg({"Price": lambda x: x.sum()}).reset_index()

# переименование столбца 
orders.rename(columns={"InvoiceDate2": "InvoiceDate"}, inplace=True)

# проверка нового датафрейма
print(len(orders))
orders.head()

22190
Out[69]:

,InvoiceNo,CustomerID,InvoiceDate,Price
0,536365,17850.0,2010-12-01,139.12
1,536366,17850.0,2010-12-01,22.20
2,536367,13047.0,2010-12-01,278.73
3,536368,13047.0,2010-12-01,70.05
4,536369,13047.0,2010-12-01,17.85


In [ ]:
# Recency (давность последней покупки)

# получение максимальной даты и прибавление к ней одного дня, затем подсчёт разницы
now = orders["InvoiceDate"].max() + timedelta(days=1)
orders["Recency"] = orders["InvoiceDate"].apply(lambda x: (now - x).days)

# альтернативный вариант (ручной):
# вывод максимальной даты и прибавление к ней одного дня, потом расчёт давности
# print(orders.InvoiceDate.max()) 
# 2011-12-09 00:00:00
# orders["Recency"] = orders.InvoiceDate.apply(lambda x: (datetime.strptime("2011-12-10", "%Y-%m-%d") - x).days)
orders.sample(7)

Out[70]:

,InvoiceNo,CustomerID,InvoiceDate,Price,Recency
11810,566018,13137.0,2011-09-08,473.29,93
18122,580667,15031.0,2011-12-05,175.53,5
16053,575899,16133.0,2011-11-11,359.90,29
11053,564060,17073.0,2011-08-22,453.79,110
8088,556876,16241.0,2011-06-15,173.82,178
16401,576672,12731.0,2011-11-16,1497.52,24
5872,551535,17615.0,2011-05-01,153.75,223


In [ ]:
# словарь с тремя функциями для агрегации при группировке
aggr = {
  "InvoiceDate": lambda x: len([d for d in x]),    # создаётся список из всех дат по заказам пользователя, считается его длина
  "Recency": lambda x: x.min(),                    # из всех дней выбирается ближайший день, т.е. минимальное значение
  "Price": lambda x: sum([p for p in x])           # суммирование
}
rfm = orders.groupby(by="CustomerID").agg(aggr).reset_index()

# переименование и сдвиг столбцов
rfm.rename(columns={"InvoiceDate": "Frequency", "Price": "Monetary"}, inplace=True)
rfm = rfm[["CustomerID", "Recency", "Frequency", "Monetary"]]
rfm.head()

Out[76]:

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,2,0.00
1,12347.0,3,7,4310.00
2,12348.0,76,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,311,1,334.40


In [ ]:
# проверка на наличие в столбцах отрицательных значений
print(rfm.query("Recency < 0"))
print(rfm.query("Frequency < 0"))
print(rfm.query("Monetary < 0"))

# print(rfm.query("Monetary < 0")["Monetary"].count())
# итог: в столбце M всё-равно почему-то присутствуют отрицательные значения - 43 строки

Empty DataFrame
Columns: [CustomerID, Recency, Frequency, Monetary]
Index: []
Empty DataFrame
Columns: [CustomerID, Recency, Frequency, Monetary]
Index: []
 CustomerID Recency Frequency Monetary
125 12503.0 338 1 -1.126000e+03
127 12505.0 302 1 -4.500000e+00
212 12605.0 366 1 -7.500000e+00
213 12607.0 59 2 -2.273737e-13
264 12666.0 360 1 -2.274400e+02
415 12870.0 367 1 -1.490000e+01
467 12943.0 302 1 -3.750000e+00
619 13154.0 145 1 -6.118600e+02
991 13672.0 302 6 -7.146000e+01
1004 13693.0 326 2 -3.200000e+01
1106 13829.0 360 1 -1.020000e+02
1200 13958.0 373 1 -1.024500e+02
1318 14119.0 355 1 -1.990000e+01
1384 14213.0 372 1 -1.192200e+03
1544 14437.0 144 2 -4.340000e+01
1642 14557.0 65 2 -1.136868e-13
1688 14627.0 312 1 -2.185000e+01
1727 14679.0 372 1 -2.550000e+00
1799 14777.0 5 2 -1.745000e+01
2236 15369.0 145 1 -1.592490e+03
2400 15590.0 290 1 -3.019000e+01
2436 15638.0 302 1 -9.400000e+01
2506 15728.0 50 1 -1.348000e+02
2560 15802.0 143 3 -4.514200e+02
2578 15823.0 338 2 -8.407600e+02
2745 16061.0 270 1 -2.995000e+01
2802 16138.0 369 1 -7.950000e+00
2892 16252.0 367 1 -2.950900e+02
2900 16262.0 361 1 -3.165000e+01
3020 16428.0 82 1 -2.950000e+00
3039 16454.0 45 3 -8.150000e+00
3103 16546.0 291 3 -9.593000e+01
3126 16579.0 366 1 -3.060000e+01
3435 16995.0 373 1 -1.250000e+00
3487 17065.0 144 1 -1.123500e+02
3650 17307.0 366 1 -1.526400e+02
3756 17448.0 145 1 -4.287630e+03
3828 17547.0 373 1 -1.180000e+01
3829 17548.0 218 3 -1.414800e+02
3870 17603.0 51 5 -1.165300e+03
4029 17820.0 192 2 -2.405000e+01
4261 18141.0 361 1 -3.540000e+01
4350 18256.0 355 1 -5.010000e+01

In [ ]:
# вычисление квантилей
quintiles = rfm[["Recency", "Frequency", "Monetary"]].quantile([.2, .4, .6, .8]).to_dict()
quintiles

Out[78]: {'Recency': {0.2: 12.0, 0.4: 32.0, 0.6: 72.0, 0.8: 179.0},
 'Frequency': {0.2: 1.0, 0.4: 2.0, 0.6: 4.0, 0.8: 7.0},
 'Monetary': {0.2: 234.39200000000005,
 0.4: 465.41200000000003,
 0.6: 909.134,
 0.8: 2002.0920000000003}}

In [ ]:
# функция для присвоения каждому пользователю нужной группы (ранга)
def score_rfm(x, c):
  rank = [1, 2, 3, 4, 5]
  if c == "Recency":
    rank = rank[::-1]     # для Recency (Давности) нужно "развернуть" ранжирование
    
  if x <= quintiles[c][.2]:
    return rank[0]
  elif x <= quintiles[c][.4]:
    return rank[1]
  elif x <= quintiles[c][.6]:
    return rank[2]
  elif x <= quintiles[c][.8]:
    return rank[3]
  else:
    return rank[4]

In [ ]:
# присвоение рангов
rfm["R"] = rfm["Recency"].apply(lambda x: score_rfm(x, "Recency"))
rfm["F"] = rfm["Frequency"].apply(lambda x: score_rfm(x, "Frequency"))
rfm["M"] = rfm["Monetary"].apply(lambda x: score_rfm(x, "Monetary"))
rfm["RFM"] = rfm["R"].map(str) + rfm["F"].map(str) + rfm["M"].map(str)
rfm.head()

Out[80]:

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM
0,12346.0,326,2,0.00,1,2,1,121
1,12347.0,3,7,4310.00,5,4,5,545
2,12348.0,76,4,1797.24,2,3,4,234
3,12349.0,19,1,1757.55,4,1,4,414
4,12350.0,311,1,334.40,1,1,2,112


In [ ]:
# карта (словарь) для сегментации пользователей
segm_map = {
  r"[1-2][1-2][1-5]": "hibernating",
  r"[1-2][3-4][1-5]": "at risk",
  r"[1-2][5][1-5]": "can\'t loose",
  r"[3][1-2][1-5]": "about to sleep",
  r"[3][3][1-5]": "need attention",
  r"[3-4][4-5][1-5]": "loyal customers",
  r"[4][1][1-5]": "promising",
  r"[5][1][1-5]": "new customers",
  r"[4-5][2-3][1-5]": "potential loyalists",
  r"[5][4-5][1-5]": "champions"
}
# параметр "Monetary" не используется для определения пользователя в тот или иной сегмент
# поэтому третья цифра исключена (одинакова для всех групп) - [1-5]

In [ ]:
# текстовое описание сегмента
rfm["Segment"] = rfm["RFM"].replace(segm_map, regex=True)
rfm.head()

Out[84]:

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM,Segment
0,12346.0,326,2,0.00,1,2,1,121,hibernating
1,12347.0,3,7,4310.00,5,4,5,545,champions
2,12348.0,76,4,1797.24,2,3,4,234,at risk
3,12349.0,19,1,1757.55,4,1,4,414,promising
4,12350.0,311,1,334.40,1,1,2,112,hibernating


In [ ]:
# небольшой анализ
print(f"Всего пользователей: {len(rfm)}")
print("--" * 15)
rfm.groupby(by="Segment")["Segment"].count().sort_values()

Всего пользователей: 4372
------------------------------
Out[100]: Segment
can't loose 49
new customers 60
promising 151
need attention 198
at risk 436
about to sleep 439
champions 578
potential loyalists 603
loyal customers 619
hibernating 1239
Name: Segment, dtype: int64

In [ ]:
# выгрузка данных в csv (через кнопку)
display(rfm)

In [ ]:
# ссылка на дашборд в Tableau
# https://public.tableau.com/profile/valerii.mamontov#!/vizhome/RMF_16089053326860/Dashboard